# Reconocimiento de vocales

In [ ]:
apt-get install ffmpeg

In [2]:
# transform all formats to .wav

import os 
import glob
import librosa
import soundfile

CURRET_PATH = os.getcwd()
SAMPLES_PATH = os.path.join(CURRET_PATH, 'database')
path = r"/mnt/c/Users/USUARIO/Google Drive/Education/urosario/github/macc/machine learning/Proyecto 2 - reconocimiento vocales NN/database/a/a (1).m4a"
a, sr = librosa.load(path, sr = 16000)

# for format in ['ogg', 'm4a'][1:2]:
#     for vowel in ['a', 'e', 'i', 'o', 'u'][0]:
#         # absolute path to search all text files inside a specific folder
#         path = os.path.join(SAMPLES_PATH, f'{vowel}', f'*.{format}')
#         print(path)
#         files = glob.glob(path)
#         print(files)

#         i = 0
#         for file in files:
#             audio = librosa.load(file)
#             soundfile.write(f'{vowel}t_({i}).wav', audio ,16000)
#             i=+1            

#             os.remove(file)


            

/home/cgaravitoc/miniconda3/envs/market_basket/lib/python3.9/site-packages/librosa/util/decorators.py:88: UserWarning: PySoundFile failed. Trying audioread instead.
  return f(*args, **kwargs)


NoBackendError: 

# Referencias

[Sound Visualization](https://maelfabien.github.io/machinelearning/Speech10/#)

[Vibration Analysis: FFT, PSD, and Spectrogram Basics](https://blog.endaq.com/vibration-analysis-fft-psd-and-spectrogram)

[The Power Spectrum](https://mark-kramer.github.io/Case-Studies-Python/03.html)

[Formate](https://es.wikipedia.org/wiki/Formante)

[Las características acústicas de los elementos segmentales](http://liceu.uab.es/~joaquim/phonetics/fon_anal_acus/caract_acust.html): Acá hay una imagen de los espectrogramas de las vocales, en donde se identifican los formantes. 

[Audio Deep Learning Made Simple: Sound Classification, Step-by-Step](https://towardsdatascience.com/audio-deep-learning-made-simple-sound-classification-step-by-step-cebc936bbe5)

[Introduction to Audio Classification](https://www.analyticsvidhya.com/blog/2021/06/introduction-to-audio-classification/)

[Audio Data Analysis Using Deep Learning with Python (Part 1)](https://www.kdnuggets.com/2020/02/audio-data-analysis-deep-learning-python-part-1.html)